# PS3: Let's Classify RNA data using K-Nearest Neighbors (KNN)
In this problem set, we will implement the K-Nearest Neighbors (KNN) algorithm to classify RNA data. 

> __Learning Objectives:__
>
> By the end of this problem set, you should be able to:
> Three learning objectives here

Let's get started!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> __Environment Setup with Include.jl__
>
> The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/).

Let's set up our code environment:

In [1]:
include(joinpath(@__DIR__, "Include.jl")); # include the Include.jl file

In addition to standard Julia libraries, we'll also use [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl). Check out [the documentation](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/) for more information on the functions, types, and data used in this material.

### Data
Let's load [a dataset from the `LIBSVM` data archive](https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/) that describes the detection of non-coding RNA sequences that was initially published by:
* [Andrew V Uzilov, Joshua M Keegan, and David H Mathews. Detection of non-coding RNAs on the basis of predicted secondary structure formation free energy change. BMC Bioinformatics, 7(173), 2006.](https://pubmed.ncbi.nlm.nih.gov/16566836/)


Non-coding RNAs (ncRNAs) have many roles in cells. However, detecting novel ncRNAs in biochemical screens is challenging. Accurate computational methods for detecting ncRNAs in sequenced genomes are important to understanding the roles ncRNAs play in cells. 

> __What's in the dataset?__
> 
> In this dataset, there are `59535` training instances in the `training` data; each instance has `8` continuous features and a binary label $y\in\left\{-1,1\right\}$. The `test` dataset has `271617` instances (with the same `8` continuous features and a binary label).

We begin by loading the `training` and `test` datasets. The [`LIBSVM` library authors](https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/) have developed these subsets. 

However, we'll need to preprocess both the `training` and `test` sets. Before we do this, we'll set some constants. Please look at the comment next to the constant for a definition of what it is, units, permissible values, etc.

In [2]:
number_of_features = 8; # there are eight continuous features

In the code block below we preprocess the `training` dataset. We have [z-score centered](https://en.wikipedia.org/wiki/Standard_score) the training data and combined it into an array where each row is a training instance, while the first `1:number_of_features` columns hold the features. The last column has the label. 

We store the training data in the `training::Array{NamedTuple,1}` array:

In [3]:
training = let

    # load the training data -
    data = parser(joinpath(_PATH_TO_DATA, "cod-rna-training.data"));
    number_of_rows = size(data,1);
    data_perm = randperm(number_of_rows);
    training_data = Array{NamedTuple,1}(undef, number_of_rows);
    X = data[data_perm,:];
    
    # z-score center the data -
    μ = mean(X[:,1:number_of_features],dims=1);
    σ = std(X[:,1:number_of_features],dims=1);
    X̂ = zeros(number_of_rows,number_of_features+1);
    for i ∈ 1:number_of_rows
        for j ∈ 1:number_of_features
            X̂[i,j] = (X[i,j] - μ[j])/(σ[j]);
        end
        X̂[i,end] = X[i,end]; # get the label
    end

    # package the data into an array of named tuples -
    for i ∈ 1:number_of_rows
        features = X̂[i,1:number_of_features];
        label = X̂[i,end];
        training_data[i] = (x = features, y = (label |> Int ));
    end

    training_data; # return scaled - balanced data
end;

Next, we preprocess the `test` dataset.  We [z-score center](https://en.wikipedia.org/wiki/Standard_score) the test data and combined it into an array where each row is a test instance, while the first `1:number_of_features` columns hold the features. The last column has the label.

We store the training data in the `test::Array{NamedTuple,1}` array:

In [4]:
test = let

    # load the training data -
    data = parser(joinpath(_PATH_TO_DATA, "cod-rna-testing.data"));
    number_of_rows = size(data,1);
    data_perm = randperm(number_of_rows);
    test_data = Array{NamedTuple,1}(undef, number_of_rows);
    X = data[data_perm,:];
    
    # z-score center the data -
    μ = mean(X[:,1:number_of_features],dims=1);
    σ = std(X[:,1:number_of_features],dims=1);
    X̂ = zeros(number_of_rows,number_of_features+1);
    for i ∈ 1:number_of_rows
        for j ∈ 1:number_of_features
            X̂[i,j] = (X[i,j] - μ[j])/(σ[j]);
        end
        X̂[i,end] = X[i,end]; # get the label
    end

    # package the data into an array of named tuples -
    for i ∈ 1:number_of_rows
        features = X̂[i,1:number_of_features];
        label = X̂[i,end];
        test_data[i] = (x=features, y = (label |> Int ));
    end

    test_data; # return scaled - balanced data
end;

__Are the labels in the training dataset balanced?__ Since we are using a KNN classifier, it is important to have balanced labels in the training dataset. If the labels are imbalanced in our reference dataset, the KNN classifier may be biased towards the majority class, which can lead to poor performance on the minority class. We can check for label balance by looking at the distribution of labels in the training datasets.

In [5]:
let 

    # initialize -
    D = training; # specify what data set we are working with
    number_of_training_examples = length(D); # how many examples are in the training dataset?

    # Fancy! Let's use a list comprehension to count the number of positive and negative labels in the training dataset. The `sum()` function will sum up the number of times the condition is true for each example in the training dataset.
    count_positive_labels = sum(D[i].y == 1 for i ∈ 1:number_of_training_examples); # how many positive labels are in the training dataset?
    count_negative_labels = sum(D[i].y == -1 for i ∈ 1:number_of_training_examples); # how many negative labels are in the training dataset?
    
    # print the results (fraction of positive and negative labels in the training dataset)
    println("Fraction of positive labels in the training dataset: ", count_positive_labels / number_of_training_examples);
    println("Fraction of negative labels in the training dataset: ", count_negative_labels / number_of_training_examples);
end

Fraction of positive labels in the training dataset: 0.3333333333333333
Fraction of negative labels in the training dataset: 0.6666666666666666


___

## Task 2: Let's look at our KNN Classifier
In this task, we will build and evaluate the KNN classifier that we will be using to make predictions on the test dataset. We'll build [a `MyWeightedKernelizedKNNClassificationModel` model](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/binaryclassification/#VLDataScienceMachineLearningPackage.MyWeightedKernelizedKNNClassificationModel) using the `training` dataset as the reference data.

Let's build a kernel function $k:\mathbb{R}^{m}\times\mathbb{R}^{m}\to\mathbb{R}$ to measure similarity. For now, let's make up our own kernel function, and save this function in the `k(x,y)::Function` variable.

In [6]:
k(x,y,γ) = exp(-γ * norm(x-y,2)^2) # RBF kernel

k (generic function with 1 method)

#### Check: Are we using a valid Kernel function?
Let's check to see if the distance (similarity) metric we built is a valid kernel function.

> __Condition:__
>
> A function $k:\mathbb{R}^{m}\times\mathbb{R}^{m}\to\mathbb{R}$ is a _valid kernel function_ if and only if the kernel matrix $\mathbf{K}\in\mathbb{R}^{n\times{n}}$ is positive (semi)definite for all possible choices of the data vectors $\mathbf{v}_i$, where $K_{ij} = k(\mathbf{v}_i, \mathbf{v}_j)$. If $\mathbf{K}$ is positive (semi)definite, then for any real-valued vector $\mathbf{x} \in \mathbb{R}^n$, the kernel matrix $\mathbf{K}$ must satisfy $\mathbf{x}^{\top}\mathbf{K}\mathbf{x} \geq 0$. 

Let's compute the kernel matrix `KM::Array{Float64,2}` for a data matrix `X::Array{Float64,2}` using the distance/kernel function `k(x,y)::Function` we built above.

In [ ]:
KM = let

    D = training; # specify what data set we are working with
    number_of_training_examples = 1000; # how many examples are in the training dataset (we will use only small number examples to compute the kernel matrix for computational efficiency)
    number_of_features = D[1].x |> length; # number of features in the dataset
    γ = 0.5; # kernel parameter

    # fill up the feature matrix -
    X = zeros(number_of_training_examples, number_of_features); # initialize a matrix to hold the features
    for i ∈ 1:number_of_training_examples
        X[i,:] = D[i].x; # fill the matrix with the features from the training dataset
    end

    # fill up the kernel matrix -
    K = zeros(number_of_training_examples,number_of_training_examples);
    for i ∈ 1:number_of_training_examples
        vᵢ = X[i,:];
        for j ∈ 1:number_of_training_examples
            vⱼ = X[j,:];
            K[i,j] = k(vᵢ,vⱼ,γ) # compute kernel value
        end
    end
    K
end;

Next, let's check to see if the kernel matrix `K::Array{Float64,2}` is positive (semi)definite by checking if all of its eigenvalues are non-negative.

> __Check:__
>
> For this kernel to be valid, the kernel matrix $\mathbf{K}$ needs to be positive (semi)definite, i.e., all eigenvalues $\lambda_i \geq 0$. We compute the eigenvalues using [`eigvals`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigvals) and verify they are all non-negative using [the `@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with [the `all` function](https://docs.julialang.org/en/v1/base/collections/#Base.all-Tuple%7BAny%7D).

Do we blow up? If not, the matrix is PSD for this dataset, which supports using this kernel in this lab.

In [ ]:
let
    λ = eigvals(K);
    @assert all(λ .≥ -1e-10) "Kernel matrix is not PSD: min eigenvalue = $(minimum(λ))"
end

## Summary
One concise summary sentence of the problem set here.

> __Key Takeaways:__
>
> Three key takeaways here

One direct, concise conclusion sentence here.
___